In [1]:
from configparser import ConfigParser
import os
from pathlib import Path
import shutil

import vak

In [2]:
TWEETYNET_CONFIGS = Path('/home/bart/Documents/repos/birdsong/tweetynet/src/configs')
BR_CONFIGS = sorted(list(TWEETYNET_CONFIGS.glob('*BirdsongRecognition*ini')))

BR_DATA = Path('/media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined/')

In [3]:
BR_CONFIGS = [path for path in BR_CONFIGS if 'bird07' not in str(path)]

get rid of existing test directories

In [10]:
BR_DATA = Path('/media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined/')

for test_dir in BR_DATA.glob('Bird*/learning_curve*/test/'):
    shutil.rmtree(str(test_dir))

"move" `VocalizationDataset`

In [7]:
for bird_dir in BR_DATA.glob('Bird*'):
    for split in ['train', 'test']:
        vds_path = list(bird_dir.glob(f'*{split}.vds.json'))[0]
        vds = vak.dataset.VocalizationDataset.load(str(vds_path))
        for voc in vds.voc_list:
            new_spect_path = str(bird_dir.joinpath(
                Path(voc.spect_path).parents[0].stem, Path(voc.spect_path).name
            ))
            assert os.path.isfile(new_spect_path)
            voc.spect_path = new_spect_path
        vds.save(vds_path)

fix paths in Tensorflow checkpoint files

In [8]:
checkpoints = list(BR_DATA.glob(
    'Bird*/learning_curve*/train/records_for_training_set_with_duration_of_*_sec_replicate_*/TweetyNet/'
    'checkpoints/checkpoint'
))
for checkpoint in checkpoints:
    with open(checkpoint, 'r') as fp:
        new_lines = []
        lines = fp.readlines()
        for line in lines:
            if '/home/art/Documents/data/birdsong/BirdsongRecognition/vak' in line:
                new_line = line.replace('/home/art/Documents/data/birdsong/BirdsongRecognition/vak',
                                        '/media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined')
            elif '/home/nickledave/Documents/data/BirdsongRecognition/vak' in line:
                new_line = line.replace('/home/nickledave/Documents/data/BirdsongRecognition/vak',
                                        '/media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined')
            elif '/home/bart/Documents/data/birdsong/vak' in line:
                new_line = line.replace('/home/bart/Documents/data/birdsong/vak',
                                        '/media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined')
            else:
                new_line = line

            # fix 'inserted' data for Bird0
            if 'learning_curve.190624_142952' in new_line:
                new_line = new_line.replace('learning_curve.190624_142952', 'learning_curve.190616_004340')
            # fix 'inserted' data for Bird1
            elif 'learning_curve.190624_183408' in new_line:
                new_line = new_line.replace('learning_curve.190624_183408', 'learning_curve.190616_063634')

            new_lines.append(new_line)
        lines = new_lines

    with open(checkpoint, 'w') as fp:
        for line in lines:
            fp.write(line)

rerun test

In [11]:
for config_file in BR_CONFIGS:
    config_obj = ConfigParser()
    config_obj.read(config_file)

    for key in ['train_vds_path', 'val_vds_path', 'test_vds_path']:
        p = Path(config_obj['TRAIN'][key])
        new_path = str(BR_DATA.joinpath(p.parent.stem, p.name))
        config_obj['TRAIN'][key] = new_path

    p = Path(config_obj['OUTPUT']['root_results_dir'])
    new_path = str(BR_DATA.joinpath(p.name))
    config_obj['OUTPUT']['root_results_dir'] = new_path

    p = Path(config_obj['OUTPUT']['results_dir_made_by_main_script'])
    new_path = str(BR_DATA.joinpath(p.parent.stem, p.name))
    config_obj['OUTPUT']['results_dir_made_by_main_script'] = new_path
    
    train_config = vak.config.parse_train_config(config_obj, config_file)
    nets_config = vak.config.parse._get_nets_config(config_obj, train_config.networks)
    output_config = vak.config.parse_output_config(config_obj)

    vak.core.learncurve.test(results_dirname=output_config.results_dirname,
                             test_vds_path=train_config.test_vds_path,
                             train_vds_path=train_config.train_vds_path,
                             networks=nets_config,
                             train_set_durs=[60, 120, 480],
                             num_replicates=train_config.num_replicates,
                             normalize_spectrograms=train_config.normalize_spectrograms,
                             save_transformed_data=False)

I0625 09:05:21.234946 140531209651968 test.py:89] Logging run of learncurve.test to '/media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined/Bird0/learning_curve.190616_004340/test'
I0625 09:05:21.235756 140531209651968 test.py:94] Loading training VocalizationDataset from /media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined/Bird0


[########################################] | 100% Completed | 17.6s


I0625 09:05:39.794826 140531209651968 test.py:117] Size of each timebin in spectrogram, in seconds: {timebin_dur}
I0625 09:05:39.795687 140531209651968 test.py:126] Loading test VocalizationDataset from /media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined/Bird0


[########################################] | 100% Completed |  7.1s


I0625 09:05:47.842578 140531209651968 test.py:189] getting train and test error for training set with duration of 60 seconds, replicate 1
W0625 09:05:48.351847 140531209651968 deprecation_wrapper.py:119] From /home/bart/Documents/repos/birdsong/tweetynet/src/tweetynet/model.py:164: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0625 09:05:48.354247 140531209651968 deprecation_wrapper.py:119] From /home/bart/Documents/repos/birdsong/tweetynet/src/tweetynet/model.py:45: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0625 09:05:48.356482 140531209651968 deprecation.py:323] From /home/bart/Documents/repos/birdsong/tweetynet/src/tweetynet/model.py:227: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0625 09:05:48.360753 140531209651968 deprecation.py:506] From /home/bart/anaconda3/env

...

[########################################] | 100% Completed | 16.2s


I0625 09:25:33.753564 140531209651968 test.py:117] Size of each timebin in spectrogram, in seconds: {timebin_dur}
I0625 09:25:33.754501 140531209651968 test.py:126] Loading test VocalizationDataset from /media/bart/HD-LCU3/tweetynet_paper/BirdsongRecognition_copy_combined/Bird9


[########################################] | 100% Completed |  7.1s


I0625 09:25:42.266303 140531209651968 test.py:189] getting train and test error for training set with duration of 60 seconds, replicate 1
I0625 09:25:44.436894 140531209651968 test.py:309] calculating training set error
I0625 09:25:45.243186 140531209651968 test.py:328] train error was 0.0016666666666666668
I0625 09:25:45.250195 140531209651968 test.py:352] Levenshtein distance for train set was 3
I0625 09:25:45.258205 140531209651968 test.py:357] Syllable error rate for train set was 0.007518796992481203
I0625 09:25:45.258677 140531209651968 test.py:362] calculating test set error
I0625 09:25:50.039003 140531209651968 test.py:381] test error was 0.033488749834880156
I0625 09:25:50.172084 140531209651968 test.py:402] Levenshtein distance for test set was 490
I0625 09:25:50.303600 140531209651968 test.py:407] Syllable error rate for test set was 0.17831149927219797
I0625 09:25:50.313923 140531209651968 test.py:189] getting train and test error for training set with duration of 60 second

I0625 09:27:12.722980 140531209651968 test.py:357] Syllable error rate for train set was 0.019435736677115987
I0625 09:27:12.723640 140531209651968 test.py:362] calculating test set error
I0625 09:27:17.543025 140531209651968 test.py:381] test error was 0.02566084629422154
I0625 09:27:17.660763 140531209651968 test.py:402] Levenshtein distance for test set was 338
I0625 09:27:17.772645 140531209651968 test.py:407] Syllable error rate for test set was 0.12299854439592431
I0625 09:27:17.782508 140531209651968 test.py:189] getting train and test error for training set with duration of 480 seconds, replicate 3
I0625 09:27:20.674396 140531209651968 test.py:309] calculating training set error
I0625 09:27:26.436526 140531209651968 test.py:328] train error was 0.0037083333333333334
I0625 09:27:26.573450 140531209651968 test.py:352] Levenshtein distance for train set was 64
I0625 09:27:26.706563 140531209651968 test.py:357] Syllable error rate for train set was 0.020075282308657464
I0625 09:27: